In [1]:
import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from catboost import CatBoostClassifier
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, StandardScaler
from sklearn.metrics import auc, accuracy_score, precision_score, recall_score, roc_curve, precision_recall_curve

import pandas as pd
from tqdm import tqdm
from pyarrow import csv
import pyarrow as pa

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import plotly.graph_objects as go

from catboost import CatBoostClassifier
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
def get_data(filename, scaler=None, useScaler=False):
    df = pd.read_csv(filename)
    Y = df.Label
    df = df.drop(['Unnamed: 0', 'Label'], axis=1)

    df['Attack'] = df['Attack'].astype("string")

    X_train, X_test, y_train, y_test = train_test_split(df, Y, test_size=0.3,stratify=df.Attack, random_state=42)
    del df
    del Y

    attack_train = X_train['Attack']
    X_train = X_train.drop('Attack', axis=1)
    attack_test = X_test['Attack']
    X_test = X_test.drop('Attack', axis=1)

    X_train = X_train.drop(['SimillarHTTP'], axis=1)
    X_train[X_train['Flow Bytes/s'] == np.inf] = 0
    X_train[X_train[' Flow Packets/s'] == np.inf] = 0

    X_test = X_test.drop(['SimillarHTTP'], axis=1)
    X_test[X_test['Flow Bytes/s'] == np.inf] = 0
    X_test[X_test[' Flow Packets/s'] == np.inf] = 0

    
    if useScaler:
        X_train = scaler.transform(X_train)
    else:
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    return X_train, X_test, y_train, y_test, attack_train, attack_test, scaler

In [3]:
X_train, X_test, y_train, y_test, attack_train, attack_test, scaler = get_data('triple.csv')

/tmp/ipykernel_7073/3331398274.py:2: DtypeWarning: Columns (62,109) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


In [4]:
save(scaler, "scaler.pckl")

'scaler.pckl)' was not found in history, as a file, url, nor in the user namespace.


In [5]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

class TestData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    
class modelDNN(nn.Module):
    def __init__(self):
        super(modelDNN, self).__init__()        
        self.layer_1 = nn.Linear(111, 4096) 
        self.layer_2 = nn.Linear(4096, 2048)
        self.layer_3 = nn.Linear(2048, 64) 
        self.layer_4 = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(4096)
        self.batchnorm2 = nn.BatchNorm1d(2048)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.dropout(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.relu(self.layer_3(x))
        x = self.batchnorm3(x)
        x = self.dropout(x)
        x = self.layer_4(x)
        
        return x
    
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc *= 100
    
    return acc

In [6]:
device = torch.device("cuda:0")

In [7]:
train_data = TrainData(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
test_data = TestData(torch.FloatTensor(X_test))

In [8]:
model = modelDNN().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [9]:
        
train_loader = DataLoader(dataset=train_data, batch_size=512, shuffle=True)
epochs = 20

In [10]:
del X_train, attack_train, attack_test, scaler, y_train

In [11]:
for i in range(8):
    print("y_pred_"+str(i))

y_pred_0
y_pred_1
y_pred_2
y_pred_3
y_pred_4
y_pred_5
y_pred_6
y_pred_7


In [12]:
import pickle

def save(data, filename):
    file = open(filename, 'wb')
    pickle.dump(data, file)
    file.close()

In [13]:
for e in range(epochs):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()

        y_pred = model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    model.eval()
    
    for i in range(8):
        del y_pred
        y_pred = model(torch.Tensor(X_test[i*100000 : min(i*100000 + 100000, X_test.shape[0])]).to(device))
        save(y_pred, "y_pred_"+str(i))
        test_acc = binary_acc(y_pred, torch.Tensor(np.array(y_test[i*100000 : min(i*100000 + 100000, y_test.shape[0])])).to(device).unsqueeze(1)) 
        print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Train Acc: {epoch_acc/len(train_loader):.3f} | Test Acc: {test_acc:.3f}')
    print("-----------------------------------------------------------------------------------------------------------------------\n")

Epoch 000: | Loss: 0.02031 | Train Acc: 99.333 | Test Acc: 99.480
Epoch 000: | Loss: 0.02031 | Train Acc: 99.333 | Test Acc: 99.489
Epoch 000: | Loss: 0.02031 | Train Acc: 99.333 | Test Acc: 99.486
Epoch 000: | Loss: 0.02031 | Train Acc: 99.333 | Test Acc: 99.512
Epoch 000: | Loss: 0.02031 | Train Acc: 99.333 | Test Acc: 99.549
Epoch 000: | Loss: 0.02031 | Train Acc: 99.333 | Test Acc: 99.477
Epoch 000: | Loss: 0.02031 | Train Acc: 99.333 | Test Acc: 99.514
Epoch 000: | Loss: 0.02031 | Train Acc: 99.333 | Test Acc: 99.445
-----------------------------------------------------------------------------------------------------------------------

Epoch 001: | Loss: 0.01598 | Train Acc: 99.484 | Test Acc: 99.448
Epoch 001: | Loss: 0.01598 | Train Acc: 99.484 | Test Acc: 99.468
Epoch 001: | Loss: 0.01598 | Train Acc: 99.484 | Test Acc: 99.450
Epoch 001: | Loss: 0.01598 | Train Acc: 99.484 | Test Acc: 99.482
Epoch 001: | Loss: 0.01598 | Train Acc: 99.484 | Test Acc: 99.522
Epoch 001: | Loss: 0.

KeyboardInterrupt: 

In [17]:
save(y_test, "y_test")

In [15]:
torch.save(model, "new-KNN.torch")

In [28]:
pickle.dump(model, file)

In [29]:
file.close()

In [31]:
file = open('X_test.data', 'wb')
pickle.dump(X_test, file)
file.close()

In [33]:
file = open('y_test.data', 'wb')
pickle.dump(y_test, file)
file.close()

In [32]:
y_test

1856763    0
712274     0
1483074    0
101751     0
1853696    0
          ..
2442436    0
1149877    0
675851     0
876316     0
1791289    0
Name: Label, Length: 746167, dtype: int64

In [30]:
model

modelDNN(
  (layer_1): Linear(in_features=111, out_features=4096, bias=True)
  (layer_2): Linear(in_features=4096, out_features=2048, bias=True)
  (layer_3): Linear(in_features=2048, out_features=1024, bias=True)
  (layer_4): Linear(in_features=1024, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [25]:
X_test

array([[0.        , 0.        , 0.        , ..., 0.08838295, 0.15467489,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.08838295, 0.15467489,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.1767659 , 0.13257847,
        0.5       ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.11047869, 0.08838565,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.5       ],
       [0.        , 0.        , 0.        , ..., 0.15467017, 0.1767713 ,
        0.5       ]])

In [59]:
class tabularCNN(nn.Module):
    def __init__(self):
        super(tabularCNN, self).__init__()
        self.dense_1 = nn.Linear(111, 1024)
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm_1 = nn.BatchNorm1d(1024)
        
        self.conv_1 = nn.Conv1d(64, 32, 5)
        
        self.relu = nn.ReLU()
        
        self.dense_2 = nn.Linear(2048, 1024)
        self.batchnorm_2 = nn.BatchNorm1d(1024)
        self.dense_3 = nn.Linear(1024, 128)
        self.batchnorm_3 = nn.BatchNorm1d(128)
        self.dense_4 = nn.Linear(128, 1)
        
        
    def forward(self, inputs):
        x = self.relu(self.dense_1(inputs))
        x = self.batchnorm_1(x)
        x = self.dropout(x)
        
        x = x.reshape(len(x), 64, 16)
        x = self.relu(self.conv_1(x))
        
        print(x.shape)
        
        x = x.reshape(len(x), 2048)
        x = self.relu(self.dense_2(x))
        x = self.batchnorm_2(x)
        x = self.dropout(x)
        x = self.relu(self.dense_3(x))
        x = self.batchnorm_3(x)
        x = self.dropout(x)
        x = self.relu(self.dense_4(x))
        return x

In [60]:
model = tabularCNN().to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)
        
train_loader = DataLoader(dataset=train_data, batch_size=16384, shuffle=True)
epochs = 10

In [61]:
for e in range(epochs):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()

        y_pred = model(X_batch)

        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
    model.eval()
    
    for i in range(80):
        del y_pred
        y_pred = model(torch.Tensor(X_test[i*10000 : min(i*10000 + 10000, X_test.shape[0])]).to(device))
        save(y_pred, "y_pred_"+str(e)+"+"+str(i))
        test_acc = binary_acc(y_pred, torch.Tensor(np.array(y_test[i*10000 : min(i*10000 + 10000, y_test.shape[0])])).to(device).unsqueeze(1)) 
        print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Train Acc: {epoch_acc/len(train_loader):.3f} | Test Acc: {test_acc:.3f}')
    print("-----------------------------------------------------------------------------------------------------------------------\n")

torch.Size([16384, 32, 12])


RuntimeError: shape '[16384, 2048]' is invalid for input of size 6291456

In [19]:
model

modelDNN(
  (layer_1): Linear(in_features=111, out_features=4096, bias=True)
  (layer_2): Linear(in_features=4096, out_features=2048, bias=True)
  (layer_3): Linear(in_features=2048, out_features=1024, bias=True)
  (layer_4): Linear(in_features=1024, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [20]:
torch.save(model, "TabularCNN.torch")